In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

2025-04-17 10:31:22.512077: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-17 10:31:22.562420: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-17 10:31:22.562450: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-17 10:31:22.563996: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 10:31:22.573483: I tensorflow/core/platform/cpu_feature_guar

GPU is available


In [2]:
# Read from CSV instead of pickle
dataset = pd.read_csv('cleaned_profiles.csv')
dataset

,Intro,Full Name,Workplace,Location,Connections,Photo,Followers,About,Experiences,Number of Experiences,...,Number of Scores,Languages,Number of Languages,Organizations,Number of Organizations,Interests,Number of Interests,Activities,Number of Activities,Label
0,"{'Full Name': 'chenxia (polly) Pei', 'Workplac...",chenxia (polly) Pei,Jiangsu Junyao mainly offer services to cement...,"Wuxi, Jiangsu, China",500,No,717,NaN,"{'0': {'Role': 'Sales', 'Workplace': 'Jiangsu ...",2,...,0,{},0,{},0,"{'0': {'Interest': 'Trina Solar', 'ID': '69648...",4,{'chenxia-pei-80177594': {'Full Name': 'chenxi...,1,1
1,"{'Full Name': 'NEHA CHANDOK', 'Workplace': 'So...",NEHA CHANDOK,Software Analyst,"Noida, Uttar Pradesh, India",500,No,1340,NaN,"{'0': {'Role': 'Software Analyst', 'Workplace'...",1,...,0,"{'Language 0': 'Bangla', 'Language 1': 'Englis...",3,{},0,"{'0': {'Interest': 'Mohamed El-Erian', 'ID': '...",8,{},0,1
2,"{'Full Name': 'Mounika Mungamuri', 'Workplace'...",Mounika Mungamuri,Senior Consultant at Infosys,"Hyderabad, Telangana, India",7,Yes,7,NaN,"{'0': {'Role': 'Senior Consultant', 'Workplace...",2,...,0,{},0,{},0,{},0,{},0,1
3,"{'Full Name': 'Katarina Djuric', 'Workplace': ...",Katarina Djuric,--,"Belgrade, Serbia",0,Yes,0,NaN,"{'0': {'Role': 'Instructor', 'Workplace': 'GE'...",1,...,0,{},0,{},0,"{'0': {'Interest': 'GE', 'ID': '1015', 'Catego...",1,{},0,1
4,"{'Full Name': 'Rachel Lally', 'Workplace': '--...",Rachel Lally,--,"Dublin, County Dublin, Ireland",61,Yes,61,NaN,"{'0': {'Role': 'Bartender', 'Workplace': ""O'Su...",1,...,1,{},0,{},0,"{'0': {'Interest': 'Richard Branson', 'ID': 'r...",3,"{'garyltravis': {'Full Name': 'Gary Travis', '...",6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,"{'Full Name': 'Emily Williams', 'Workplace': ""...",Emily Williams,Market Research Analyst at L'Oreal,"New York City, New York",106,No,717,{},Market Research Analyst Elizabeth Arden Jan 20...,3,...,0,{},0,{},0,{},3,{},4,11
3596,"{'Full Name': 'Sarah Johnson', 'Workplace': 'D...",Sarah Johnson,Director of Marketing Strategy at Acme Inc.,"San Francisco, California",102,No,6,{},Director of Marketing Acme Inc. January 2018-P...,5,...,0,{},0,{},0,{},5,{},3,11
3597,"{'Full Name': 'Sarah Johnson', 'Workplace': 'M...",Sarah Johnson,Manager at Acme Inc.,"New York City, New York",435,No,10,A results-driven and experienced professional ...,{},0,...,0,{},0,{},0,{},6,{},0,11
3598,"{'Full Name': 'Emma Williams', 'Workplace': 'M...",Emma Williams,Manager and Director at ABC Inc.,"Seattle, Washington",280,No,34,With over 10 years of experience in operations...,Operations Manager ABC Inc. January 2015- Pres...,1,...,0,English Spanish,2,{},0,{},4,{},4,11


In [3]:
import logging
import time
import ast
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from transformers import AutoTokenizer, AutoModel, RobertaTokenizer, RobertaModel
import gensim.downloader as gensim_downloader
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings
from flair.data import Sentence

# Simple generated suffix for file names
FILE_SUFFIX = ""

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Create output directories with suffix
output_dir = f'embeddings_output'
plots_dir = f'variance_plots'
os.makedirs(output_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

# Define numerical fields (used in multiple places)
NUMERICAL_FIELDS = [
    "Number of Experiences", "Number of Educations", "Number of Licenses",
    "Number of Volunteering", "Number of Skills", "Number of Recommendations",
    "Number of Projects", "Number of Publications", "Number of Courses",
    "Number of Honors", "Number of Scores", "Number of Languages",
    "Number of Organizations", "Number of Interests", "Number of Activities",
    "Connections", "Followers"
]

# Column names for numerical features (used in output dataframes)
NUMERICAL_FIELD_NAMES = [
    "num_experiences", "num_educations", "num_licenses",
    "num_volunteering", "num_skills", "num_recommendations",
    "num_projects", "num_publications", "num_courses",
    "num_honors", "num_scores", "num_languages",
    "num_organizations", "num_interests", "num_activities",
    "connections", "followers"
]

# Model configurations
model_configs = [
    {
        "name": "flair",
        "model_name": "en",  # Flair embeddings for English (300d)
        "output_file": os.path.join(output_dir, f'flair_PCA_scaled_embeddings{FILE_SUFFIX}.csv'),
        "temp_file": os.path.join(output_dir, f'flair_temp_embeddings{FILE_SUFFIX}.pkl'),
        "variance_plot": os.path.join(plots_dir, f'flair_explained_variance{FILE_SUFFIX}.png')
    },
    {
        "name": "glove",
        "model_name": "glove.6B.300d.txt",  # GloVe embeddings (300d)
        "output_file": os.path.join(output_dir, f'glove_PCA_scaled_embeddings{FILE_SUFFIX}.csv'),
        "temp_file": os.path.join(output_dir, f'glove_temp_embeddings{FILE_SUFFIX}.pkl'),
        "variance_plot": os.path.join(plots_dir, f'glove_explained_variance{FILE_SUFFIX}.png')
    },
    {
        "name": "roberta",
        "model_name": "roberta-base",
        "output_file": os.path.join(output_dir, f'roberta_PCA_scaled_embeddings{FILE_SUFFIX}.csv'),
        "temp_file": os.path.join(output_dir, f'roberta_temp_embeddings{FILE_SUFFIX}.pkl'),
        "variance_plot": os.path.join(plots_dir, f'roberta_explained_variance{FILE_SUFFIX}.png')
    },
    {
        "name": "bert",
        "model_name": "bert-base-uncased",
        "output_file": os.path.join(output_dir, f'bert_PCA_scaled_embeddings{FILE_SUFFIX}.csv'),
        "temp_file": os.path.join(output_dir, f'bert_temp_embeddings{FILE_SUFFIX}.pkl'),
        "variance_plot": os.path.join(plots_dir, f'bert_explained_variance{FILE_SUFFIX}.png')
    },
    {
        "name": "modernbert",
        "model_name": "answerdotai/ModernBERT-base",
        "output_file": os.path.join(output_dir, f'modernbert_PCA_scaled_embeddings{FILE_SUFFIX}.csv'),
        "temp_file": os.path.join(output_dir, f'modernbert_temp_embeddings{FILE_SUFFIX}.pkl'),
        "variance_plot": os.path.join(plots_dir, f'modernbert_explained_variance{FILE_SUFFIX}.png')
    },
    {
        "name": "deberta",
        "model_name": "microsoft/deberta-v3-large",
        "output_file": os.path.join(output_dir, f'deberta_PCA_scaled_embeddings{FILE_SUFFIX}.csv'),
        "temp_file": os.path.join(output_dir, f'deberta_temp_embeddings{FILE_SUFFIX}.pkl'),
        "variance_plot": os.path.join(plots_dir, f'deberta_explained_variance{FILE_SUFFIX}.png')
    }
]

# PCA components to try
PCA_COMPONENTS_OPTIONS = [10, 20, 30, 50, 75, 100, 125, 150, 175, 200, 250, 300, 350, 400]

# Target explained variance threshold
TARGET_VARIANCE = 0.95

# Initialize device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Global variables for models
glove_model = None
flair_embedding = None


def safe_eval(s):
    """Safely evaluate a string to convert it to a Python object"""
    if isinstance(s, str):
        try:
            return ast.literal_eval(s)
        except:
            return s
    return s


def get_embedding_transformer(text, tokenizer, model):
    """Get embeddings for transformer-based models (BERT, RoBERTa, etc.)"""
    if not text or text.strip() == "":
        # Handle empty text by returning zeros
        with torch.no_grad():
            dummy_inputs = tokenizer("test", return_tensors='pt').to(device)
            dummy_outputs = model(**dummy_inputs)
            if hasattr(dummy_outputs, 'last_hidden_state'):
                emb_dim = dummy_outputs.last_hidden_state.shape[-1]
            elif hasattr(dummy_outputs, 'pooler_output'):
                emb_dim = dummy_outputs.pooler_output.shape[-1]
            else:
                emb_dim = 768  # default fallback
        return np.zeros(emb_dim)
    
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Different models have different output structures
    if hasattr(outputs, 'last_hidden_state'):
        # Standard transformer models like BERT, RoBERTa, DeBERTa
        return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    elif hasattr(outputs, 'pooler_output'):
        # Some models provide pooled output
        return outputs.pooler_output.squeeze().cpu().numpy()
    else:
        # Fallback - use whatever tensor is available
        for key, value in outputs.items():
            if isinstance(value, torch.Tensor):
                return value.mean(dim=1).squeeze().cpu().numpy()
    
    raise ValueError(f"Could not extract embeddings from model outputs: {outputs}")


def get_embedding_glove(text):
    """Get embeddings using GloVe model"""
    global glove_model
    
    if not text or text.strip() == "":
        return np.zeros(300)  # GloVe 6B.300d has 300 dimensions
    
    # Tokenize text into words
    words = text.lower().split()
    
    # Get embeddings for each word
    word_embeddings = []
    for word in words:
        if word in glove_model:
            word_embeddings.append(glove_model[word])
    
    # Return average of word embeddings
    if word_embeddings:
        return np.mean(word_embeddings, axis=0)
    else:
        return np.zeros(300)


def get_embedding_flair(text):
    """Get embeddings using Flair"""
    global flair_embedding
    
    if not text or text.strip() == "":
        return np.zeros(300)  # Flair "en" has 300 dimensions
    
    # Create a Flair Sentence
    sentence = Sentence(text)
    
    # Embed the sentence
    flair_embedding.embed(sentence)
    
    # Get the embedding vector
    embedding_vector = sentence.embedding.cpu().numpy()
    
    return embedding_vector


def apply_ste_transformer(content_embedding, section_tag, tokenizer, model):
    tag_embedding = get_embedding_transformer(section_tag, tokenizer, model)
    return content_embedding - tag_embedding


def apply_ste_glove(content_embedding, section_tag):
    tag_embedding = get_embedding_glove(section_tag)
    return content_embedding - tag_embedding


def apply_ste_flair(content_embedding, section_tag):
    tag_embedding = get_embedding_flair(section_tag)
    return content_embedding - tag_embedding


def process_row(index, dataset, model_name, tokenizer=None, model=None):
    """Process a single row of data, extracting text and generating embeddings"""
    start_time = time.time()

    # Extract data
    intro = safe_eval(dataset["Intro"].iloc[index])
    about = safe_eval(dataset["About"].iloc[index])
    experiences = safe_eval(dataset["Experiences"].iloc[index])

    # Process intro data
    if isinstance(intro, dict):
        full_name = intro.get("Full Name", "")
        workplace = intro.get("Workplace", "")
        location = intro.get("Location", "")
    else:
        full_name = workplace = location = str(intro)

    # Process about data
    if isinstance(about, dict):
        desc = about.get("About", "")
    else:
        desc = str(about)

    # Process experiences data
    exp_text = ""
    if isinstance(experiences, list):
        for exp in experiences:
            if isinstance(exp, dict):
                exp_text += f"{exp.get('Role', '')} {exp.get('Workplace', '')} {exp.get('Duration', '')} {exp.get('Workplace Location', '')} {exp.get('Description', '')} "
    elif isinstance(experiences, dict):
        for value in experiences.values():
            if isinstance(value, dict):
                exp_text += f"{value.get('Role', '')} {value.get('Workplace', '')} {value.get('Duration', '')} {value.get('Workplace Location', '')} {value.get('Description', '')} "

    # Extract numerical fields
    numerical_data = []
    for field in NUMERICAL_FIELDS:
        value = dataset[field].iloc[index]
        if pd.isna(value):
            numerical_data.append(0)
        elif isinstance(value, (int, float)):
            numerical_data.append(value)
        else:
            try:
                numerical_data.append(float(value))
            except:
                numerical_data.append(0)  # Default to 0 if conversion fails

    # STE Implementation based on model type
    if model_name == "glove":
        # Intro section
        intro_emb = get_embedding_glove(f"{full_name} {workplace} {location}")
        fin1 = apply_ste_glove(intro_emb, "Introduction")

        # About section
        about_emb = get_embedding_glove(desc)
        fin2 = apply_ste_glove(about_emb, "About")

        # Experiences section
        exp_emb = get_embedding_glove(exp_text)
        fin3 = apply_ste_glove(exp_emb, "Experiences")
    elif model_name == "flair":
        # Intro section
        intro_emb = get_embedding_flair(f"{full_name} {workplace} {location}")
        fin1 = apply_ste_flair(intro_emb, "Introduction")

        # About section
        about_emb = get_embedding_flair(desc)
        fin2 = apply_ste_flair(about_emb, "About")

        # Experiences section
        exp_emb = get_embedding_flair(exp_text)
        fin3 = apply_ste_flair(exp_emb, "Experiences")
    else:  # Transformer models (bert, modernbert, deberta, roberta)
        # Intro section
        intro_emb = get_embedding_transformer(f"{full_name} {workplace} {location}", tokenizer, model)
        fin1 = apply_ste_transformer(intro_emb, "Introduction", tokenizer, model)

        # About section
        about_emb = get_embedding_transformer(desc, tokenizer, model)
        fin2 = apply_ste_transformer(about_emb, "About", tokenizer, model)

        # Experiences section
        exp_emb = get_embedding_transformer(exp_text, tokenizer, model)
        fin3 = apply_ste_transformer(exp_emb, "Experiences", tokenizer, model)

    # Average the text embeddings
    text_embedding = (fin1 + fin2 + fin3) / 3

    # Store text embedding and numerical data separately
    result = {
        'text_embedding': text_embedding,
        'numerical': np.array(numerical_data)
    }

    end_time = time.time()
    execution_time = end_time - start_time
    logging.info(f"Processing time for row {index}: {execution_time:.2f} seconds")

    return result


def get_embedding_dimensions(model_name):
    """Return default embedding dimensions for different models"""
    if model_name == "glove":
        return 300  # Using 300d as specified
    elif model_name == "flair":
        return 300  # Using 300d as specified
    elif model_name == "deberta":
        return 1024  # DeBERTa large has 1024 dimensions
    else:  # bert, roberta, modernbert
        return 768


def generate_embeddings(dataset, model_config, num_rows=3600, save_interval=100):
    """Generate embeddings for the dataset using the specified model"""
    model_name = model_config["name"]
    temp_file = model_config["temp_file"]
    model_path = model_config["model_name"]
    
    print(f"Loading {model_name} model: {model_path}")
    
    tokenizer = None
    model = None
    
    # Initialize models based on model type
    global glove_model, flair_embedding
    
    if model_name == "glove":
        # Load GloVe model
        try:
            # Try to load from file first
            glove_model = {}
            with open(model_path, 'r', encoding='utf-8') as f:
                for line in f:
                    values = line.strip().split()
                    word = values[0]
                    vector = np.array([float(val) for val in values[1:]])
                    glove_model[word] = vector
            print(f"Loaded GloVe from file: {model_path}")
        except FileNotFoundError:
            # If file not found, download using gensim
            print(f"File not found, downloading GloVe...")
            glove_model = gensim_downloader.load("glove-wiki-gigaword-300")
            print("Downloaded GloVe 300d model")
    
    elif model_name == "flair":
        # Load Flair embeddings
        word_embeddings = WordEmbeddings('en')
        flair_embedding = DocumentPoolEmbeddings([word_embeddings])
        print("Loaded Flair embeddings")
    
    elif model_name == "roberta":
        # Load RoBERTa model
        tokenizer = RobertaTokenizer.from_pretrained(model_path)
        model = RobertaModel.from_pretrained(model_path)
        model = model.to(device)
        model.eval()
        print("Loaded RoBERTa model")
    
    elif model_name == "modernbert":
        # Load ModernBERT model
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModel.from_pretrained(model_path)
        model = model.to(device)
        model.eval()
        print("Loaded ModernBERT model")
    
    else:  # bert or deberta
        # Load BERT/DeBERTa model
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModel.from_pretrained(model_path)
        model = model.to(device)
        model.eval()
        print(f"Loaded {model_name} model")
    
    all_embeddings = []
    total_start_time = time.time()

    # Check if there's a partially processed file
    try:
        existing_df = pd.read_pickle(temp_file)
        start_index = len(existing_df)
        all_embeddings = existing_df
        print(f"Resuming from row {start_index} for {model_name}")
    except (FileNotFoundError, EOFError):
        start_index = 0
        print(f"Starting new embedding generation for {model_name}")

    for index in range(start_index, min(num_rows, len(dataset))):
        row_start_time = time.time()
        try:
            embeddings_dict = process_row(index, dataset, model_name, tokenizer, model)
            all_embeddings.append(embeddings_dict)
        except Exception as e:
            print(f"Error processing row {index} with {model_name}: {str(e)}")
            
            # Create a dummy embeddings dict with zeros
            emb_dim = get_embedding_dimensions(model_name)
            
            dummy_dict = {
                'text_embedding': np.zeros(emb_dim),
                'numerical': np.zeros(len(NUMERICAL_FIELDS))
            }
            all_embeddings.append(dummy_dict)

        row_end_time = time.time()
        row_execution_time = row_end_time - row_start_time
        print(f"{model_name}: Processed row {index + 1} in {row_execution_time:.2f} seconds")

        # Save progress at specified intervals
        if (index + 1) % save_interval == 0:
            pd.to_pickle(all_embeddings, temp_file)
            print(f"{model_name}: Progress saved at row {index + 1}")

    # Final save of the raw embeddings
    pd.to_pickle(all_embeddings, temp_file)
    print(f"{model_name}: All raw embeddings saved to {temp_file}")
    
    # Free up memory for transformer models
    if model_name in ["bert", "modernbert", "deberta", "roberta"]:
        del model
        del tokenizer
        torch.cuda.empty_cache()
    
    return all_embeddings


def analyze_pca_variance(embeddings_list, model_config):
    """Analyze PCA explained variance for different numbers of components"""
    # Check if variance plot already exists
    if os.path.exists(model_config["variance_plot"]):
        print(f"Variance plot already exists for {model_config['name']}, skipping plot generation")
    
    # Extract text embeddings
    text_embeddings = np.vstack([emb['text_embedding'].reshape(1, -1) for emb in embeddings_list])
    embedding_dim = text_embeddings.shape[1]
    
    # Determine maximum components based on model type
    model_name = model_config['name']
    if model_name == "glove":
        max_components = min(embedding_dim, 300)  # GloVe 300d dimensions
    elif model_name == "flair":
        max_components = min(embedding_dim, 300)  # Flair "en" has 300 dimensions
    else:
        # For transformer models, use the standard cap
        max_components = min(text_embeddings.shape[0], text_embeddings.shape[1])
        max_components = min(max_components, 400)
    
    print(f"{model_name} embedding dimension: {embedding_dim}, using max {max_components} components for PCA")
    
    # Initialize PCA with maximum number of components
    pca = PCA(n_components=max_components)
    pca.fit(text_embeddings)
    
    # Calculate cumulative explained variance
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    
    # Find number of components needed for target variance
    if np.max(cumulative_variance) < TARGET_VARIANCE:
        print(f"Warning: {model_config['name']} cannot reach {TARGET_VARIANCE*100}% variance with {max_components} components")
        print(f"Maximum achievable variance is {np.max(cumulative_variance)*100:.2f}%")
        components_for_target = max_components
    else:
        components_for_target = np.argmax(cumulative_variance >= TARGET_VARIANCE) + 1
    
    print(f"{model_config['name']}: {components_for_target} components needed for {TARGET_VARIANCE*100}% variance")
    
    # Create the plot if it doesn't exist
    if not os.path.exists(model_config["variance_plot"]):
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='-', markersize=3)
        plt.axhline(y=TARGET_VARIANCE, color='r', linestyle='--', label=f"{TARGET_VARIANCE*100}% Explained Variance")
        plt.axvline(x=components_for_target, color='g', linestyle='--', 
                    label=f"{components_for_target} components")
        
        # Add vertical lines for each PCA component option - only for valid components
        valid_component_options = [c for c in PCA_COMPONENTS_OPTIONS if c <= max_components]
        for comp in valid_component_options:
            plt.axvline(x=comp, color='gray', alpha=0.3, linestyle=':')
        
        plt.title(f'Cumulative Explained Variance: {model_config["name"].upper()}')
        plt.xlabel('Number of Components')
        plt.ylabel('Cumulative Explained Variance')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.savefig(model_config["variance_plot"])
        plt.close()
    
    # Create a dataframe with detailed variance information for all components
    variance_df = pd.DataFrame({
        'component': range(1, len(pca.explained_variance_ratio_) + 1),
        'explained_variance': pca.explained_variance_ratio_,
        'cumulative_variance': cumulative_variance
    })
    
    # Save the variance data
    variance_file = os.path.join(output_dir, f"{model_config['name']}_variance_data{FILE_SUFFIX}.csv")
    
    # Check if we need to save the variance data
    if not os.path.exists(variance_file):
        variance_df.to_csv(variance_file, index=False)
        print(f"Variance data saved to {variance_file}")
    else:
        print(f"Variance data already exists at {variance_file}")
    
    # Return variance data for further analysis
    variance_data = {
        'n_components': list(range(1, len(cumulative_variance) + 1)),
        'explained_variance': pca.explained_variance_ratio_,
        'cumulative_variance': cumulative_variance,
        'optimal_components': components_for_target
    }
    
    return variance_data, pca


def analyze_pca_variance_with_checkpoint(embeddings_list, model_config):
    """Analyze PCA explained variance with checkpoint support"""
    variance_file = os.path.join(output_dir, f"{model_config['name']}_variance_data{FILE_SUFFIX}.csv")
    
    # Check if the variance data file already exists
    if os.path.exists(variance_file):
        try:
            variance_df = pd.read_csv(variance_file)
            print(f"Found existing variance data for {model_config['name']}, skipping analysis")
            
            # Extract optimal components from existing data
            cumulative_variance = variance_df['cumulative_variance'].values
            components_for_target = np.argmax(cumulative_variance >= TARGET_VARIANCE) + 1
            
            variance_data = {
                'n_components': variance_df['component'].tolist(),
                'explained_variance': variance_df['explained_variance'].values,
                'cumulative_variance': cumulative_variance,
                'optimal_components': components_for_target
            }
            
            print(f"{model_config['name']}: {components_for_target} components needed for {TARGET_VARIANCE*100}% variance")
            
            return variance_data, None
        except Exception as e:
            print(f"Error loading existing variance data: {str(e)}")
            print("Will recompute variance analysis")
    
    # If no valid existing file, perform the analysis
    return analyze_pca_variance(embeddings_list, model_config)


def generate_pca_embeddings(embeddings_list, n_components):
    """Generate PCA-reduced embeddings with standardized numerical features"""
    # Extract text and numerical embeddings
    text_embeddings = np.vstack([emb['text_embedding'].reshape(1, -1) for emb in embeddings_list])
    numerical_features = np.vstack([emb['numerical'].reshape(1, -1) for emb in embeddings_list])
    
    # Apply PCA to reduce dimensions of text embeddings
    pca = PCA(n_components=n_components)
    reduced_text_embeddings = pca.fit_transform(text_embeddings)
    
    # Print explained variance ratio
    explained_variance = np.sum(pca.explained_variance_ratio_)
    print(f"PCA with {n_components} components explains {explained_variance:.4f} of variance")
    
    # Scale the numerical features
    print("Scaling numerical features...")
    scaler = StandardScaler()
    scaled_numerical_features = scaler.fit_transform(numerical_features)
    
    # Concatenate reduced text embeddings with scaled numerical features
    final_embeddings = np.hstack([reduced_text_embeddings, scaled_numerical_features])
    
    # Create column names for the DataFrame
    columns = [f"pca_{i}" for i in range(n_components)]
    columns.extend(NUMERICAL_FIELD_NAMES)
    
    return final_embeddings, columns, pca.explained_variance_ratio_.sum()


def generate_embeddings_for_all_components(embeddings_list, model_config):
    """Generate embeddings for all PCA component options"""
    results = []
    summary_file = os.path.join(output_dir, f"{model_config['name']}_embeddings_summary{FILE_SUFFIX}.csv")
    
    # Check model dimensionality limits
    model_name = model_config['name']
    text_embeddings = np.vstack([emb['text_embedding'].reshape(1, -1) for emb in embeddings_list])
    embedding_dim = text_embeddings.shape[1]
    
    # Define max components based on model type - only limit for GloVe and Flair
    if model_name == "glove":
        max_allowed_components = min(embedding_dim, 300)  # GloVe 300d dimensions
        print(f"GloVe has {embedding_dim} dimensions, limiting components to {max_allowed_components}")
    elif model_name == "flair":
        max_allowed_components = min(embedding_dim, 300)  # Flair 300d dimensions
        print(f"Flair has {embedding_dim} dimensions, limiting components to {max_allowed_components}")
    else:
        max_allowed_components = 400  # No specific limit for transformer models
    
    # Filter component options based on model's dimensionality
    valid_component_options = [c for c in PCA_COMPONENTS_OPTIONS if c <= max_allowed_components]
    
    if len(valid_component_options) < len(PCA_COMPONENTS_OPTIONS):
        print(f"Note: Only using components up to {max_allowed_components} for {model_name} due to dimensionality constraints")
    
    # Check if summary file exists - if so, we can resume from where we left off
    try:
        existing_results = pd.read_csv(summary_file)
        # Get the components that have already been processed
        completed_components = existing_results['n_components'].tolist()
        results = existing_results.to_dict('records')
        print(f"Found existing summary for {model_config['name']} with {len(completed_components)} completed component sizes")
    except (FileNotFoundError, pd.errors.EmptyDataError):
        completed_components = []
        print(f"No existing summary found for {model_config['name']}, starting fresh")
    
    for n_components in valid_component_options:
        # Skip if this component size has already been processed
        output_file = os.path.join(output_dir, f"{model_config['name']}_PCA_{n_components}_components{FILE_SUFFIX}.csv")
        
        if n_components in completed_components and os.path.exists(output_file):
            print(f"{model_config['name']}: Skipping {n_components} components (already processed)")
            continue
            
        print(f"{model_config['name']}: Generating embeddings with {n_components} components")
        
        # Generate embeddings with this number of components
        embeddings, column_names, variance = generate_pca_embeddings(embeddings_list, n_components)
        
        # Create DataFrame with embeddings
        embeddings_df = pd.DataFrame(embeddings, columns=column_names)
        embeddings_df.to_csv(output_file, index=False)
        
        print(f"Saved embeddings with {n_components} components to {output_file}")
        
        # Store results
        results.append({
            'model': model_config['name'],
            'n_components': n_components,
            'explained_variance': variance,
            'file_path': output_file
        })
        
        # Update the summary file after each component size to facilitate resuming
        results_df = pd.DataFrame(results)
        results_df.to_csv(summary_file, index=False)
    
    # Final save of the summary DataFrame (should be complete now)
    results_df = pd.DataFrame(results)
    results_df.to_csv(summary_file, index=False)
    
    return results_df

def main():
    """Main function to run the embedding generation and analysis pipeline"""
    # Load your dataset here
    try:
        # If you need to load the dataset from a file, uncomment and modify the following line
        # dataset = pd.read_csv('your_dataset.csv')
        
        # Create results dataframe to collect all models' results
        all_model_results = []
        
        # Track max components for each model for the comparative plot
        model_max_components = {}
        
        # Check if final combined analysis exists - if so, we can skip to the specific model needed
        combined_results_path = os.path.join(output_dir, f'pca_analysis_results{FILE_SUFFIX}.csv')
        completed_models = []
        
        if os.path.exists(combined_results_path):
            try:
                combined_results = pd.read_csv(combined_results_path)
                completed_models = combined_results['model'].unique().tolist()
                all_model_results = [
                    combined_results[combined_results['model'] == model] 
                    for model in completed_models
                ]
                print(f"Found existing combined results with completed models: {', '.join(completed_models)}")
            except Exception as e:
                print(f"Error loading existing combined results: {str(e)}")
        
        for model_config in model_configs:
            model_name = model_config['name']
            
            # Skip if this model is already fully processed
            if model_name in completed_models and os.path.exists(model_config["output_file"]):
                print(f"\n\n===== Skipping {model_name} (already fully processed) =====")
                
                # Load model's component data for the comparative plot
                try:
                    model_summary = pd.read_csv(os.path.join(output_dir, f"{model_name}_embeddings_summary{FILE_SUFFIX}.csv"))
                    model_max_components[model_name] = model_summary['n_components'].max()
                except:
                    # If we can't determine the max components, use a conservative default
                    if model_name == "glove":
                        model_max_components[model_name] = 300  # Using 300d as specified
                    elif model_name == "flair":
                        model_max_components[model_name] = 300  # Using 300d as specified
                    else:
                        model_max_components[model_name] = 400
                        
                continue
                
            print(f"\n\n===== Processing with {model_name} model =====")
            
            # Step 1: Check if embeddings already exist
            if os.path.exists(model_config["temp_file"]):
                print(f"Loading existing embeddings for {model_name} from {model_config['temp_file']}")
                try:
                    all_embeddings = pd.read_pickle(model_config["temp_file"])
                    print(f"Loaded {len(all_embeddings)} embeddings for {model_name}")
                except Exception as e:
                    print(f"Error loading existing embeddings: {str(e)}")
                    print("Generating new embeddings")
                    all_embeddings = generate_embeddings(dataset, model_config, num_rows=3600, save_interval=100)
            else:
                # Generate raw embeddings (averaged text embeddings + numerical)
                all_embeddings = generate_embeddings(dataset, model_config, num_rows=3600, save_interval=100)
            
            # Step 2: Analyze PCA variance patterns with checkpoint
            variance_data, _ = analyze_pca_variance_with_checkpoint(all_embeddings, model_config)
            
            # Get the embedding dimension for this model
            text_embeddings = np.vstack([emb['text_embedding'].reshape(1, -1) for emb in all_embeddings])
            embedding_dim = text_embeddings.shape[1]
            
            # Determine max components for this model based on dimensionality
            if model_name == "glove":
                model_max_components[model_name] = min(embedding_dim, 300)  # Using 300d as specified
            elif model_name == "flair":
                model_max_components[model_name] = min(embedding_dim, 300)  # Using 300d as specified
            else:
                model_max_components[model_name] = 400
            
            print(f"{model_name} has {embedding_dim} dimensions, using max {model_max_components[model_name]} components")
            
            # Step 3: Generate embeddings for all PCA component options (with checkpoint)
            component_results = generate_embeddings_for_all_components(all_embeddings, model_config)
            all_model_results.append(component_results)
            
            # Step 4: Check if final embeddings already exist
            if os.path.exists(model_config["output_file"]):
                print(f"{model_name}: Final embeddings already exist at {model_config['output_file']}")
            else:
                # Generate final embeddings with optimal components
                optimal_components = variance_data['optimal_components']
                print(f"Generating final embeddings with {optimal_components} components")
                final_embeddings, column_names, _ = generate_pca_embeddings(all_embeddings, optimal_components)
                
                # Save final embeddings
                embeddings_df = pd.DataFrame(final_embeddings, columns=column_names)
                embeddings_df.to_csv(model_config["output_file"], index=False)
                print(f"{model_name}: Final embeddings saved to {model_config['output_file']}")
        
        # Combine all models' results for comparative analysis
        if all_model_results:
            combined_results = pd.concat(all_model_results)
            
            # Create comparative visualizations with dimensionality limits
            comparative_plot = os.path.join(plots_dir, f'comparative_variance{FILE_SUFFIX}.png')
            if not os.path.exists(comparative_plot) or len(all_model_results) > len(completed_models):
                plt.figure(figsize=(12, 8))

                # Fix GloVe duplicate entries
                if 'glove' in combined_results['model'].unique():
                    # Find duplicate components for GloVe
                    glove_data = combined_results[combined_results['model'] == 'glove']
                    component_counts = glove_data['n_components'].value_counts()
                    duplicate_components = component_counts[component_counts > 1].index.tolist()
                    
                    # Remove first occurrence of each duplicate component
                    for comp in duplicate_components:
                        # Get index of first occurrence
                        first_index = glove_data[glove_data['n_components'] == comp].index[0]
                        # Drop it from combined_results
                        combined_results = combined_results.drop(first_index)
                
                # Find the minimum x-axis limit that accommodates all models
                min_x_limit = min(model_max_components.values()) if model_max_components else 50
                
                # Use matplotlib directly instead of seaborn to avoid pandas indexing issues# In the main() function, find the section creating comparative visualizations
                comparative_plot = os.path.join(plots_dir, f'comparative_variance{FILE_SUFFIX}.png')
                if not os.path.exists(comparative_plot) or len(all_model_results) > len(completed_models):
                    # Change this:
                    plt.figure(figsize=(12, 8))
                    
                    # To:
                    plt.figure(figsize=(5, 3.5), dpi=300)
                    
                    # Find the minimum x-axis limit that accommodates all models
                    min_x_limit = min(model_max_components.values()) if model_max_components else 50
                    
                    # Use matplotlib directly instead of seaborn to avoid pandas indexing issues
                    for model in combined_results['model'].unique():
                        # Skip gemini model if it exists
                        if model == 'gemini':
                            continue
                            
                        model_data = combined_results[combined_results['model'] == model]
                        
                        # Get the model's max components
                        max_comp = model_max_components.get(model, 400)
                        
                        # Filter data to only include components up to the model's max
                        model_data = model_data[model_data['n_components'] <= max_comp]
                        
                        x_values = model_data['n_components'].values
                        y_values = model_data['explained_variance'].values
                        
                        # Sort by n_components to ensure proper line plotting
                        indices = np.argsort(x_values)
                        x_values = x_values[indices]
                        y_values = y_values[indices]
                        
                        # Add smaller marker size and thinner lines
                        plt.plot(x_values, y_values, marker='o', markersize=2, linewidth=1, linestyle='-', label=model)
                    
                    # Update title and font sizes
                    plt.title('Explained Variance vs PCA Components', fontsize=12)
                    plt.xlabel('Number of PCA Components', fontsize=12)
                    plt.ylabel('Explained Variance', fontsize=12)
                    plt.tick_params(axis='both', labelsize=8)
                    plt.legend(fontsize=12)
                    plt.grid(True, alpha=0.3)
                    plt.tight_layout(pad=0.5)
    
                    # Save with high resolution
                    plt.savefig(comparative_plot, dpi=300, bbox_inches='tight')
                    plt.close()
                    
                    # For the common range plot, apply the same styling changes:
                    common_range_plot = os.path.join(plots_dir, f'comparative_variance_common_range{FILE_SUFFIX}.png')
                    plt.figure(figsize=(5, 3.5), dpi=300)
                    
                    for model in combined_results['model'].unique():
                        # Skip gemini model if it exists
                        if model == 'gemini':
                            continue
                            
                        model_data = combined_results[combined_results['model'] == model]
                        
                        # Filter to only include components up to the minimum x-limit
                        model_data = model_data[model_data['n_components'] <= min_x_limit]
                        
                        x_values = model_data['n_components'].values
                        y_values = model_data['explained_variance'].values
                        
                        # Sort by n_components to ensure proper line plotting
                        indices = np.argsort(x_values)
                        x_values = x_values[indices]
                        y_values = y_values[indices]
        
                        # Use smaller markers and thinner lines
                        plt.plot(x_values, y_values, marker='o', markersize=2, linewidth=1, linestyle='-', label=model)
                    
                    # Update title and font sizes
                    plt.title(f'Explained Variance (0-{min_x_limit} Components)', fontsize=10)
                    plt.xlabel('Number of PCA Components', fontsize=9)
                    plt.ylabel('Explained Variance', fontsize=9)
                    plt.ylim(0.6, 1)  # Set y-axis limit
                    plt.tick_params(axis='both', labelsize=8)
                    plt.legend(fontsize=9)
                    plt.grid(True, alpha=0.3)
                    plt.tight_layout(pad=0.5)
                    
                    # Save with high resolution
                    plt.savefig(common_range_plot, dpi=300, bbox_inches='tight')
                    plt.close()
                

                
                # Save the combined results
                combined_results.to_csv(combined_results_path, index=False)
        
    except NameError:
        print("Error: 'dataset' variable not found. Please load your dataset before running this script.")
        print("For example, add the following line before calling main():")
        print("dataset = pd.read_csv('your_dataset.csv')")
    
    print(f"All processing complete. Files saved with suffix: {FILE_SUFFIX}")
    print(f"Output directories: {output_dir} and {plots_dir}")

if __name__ == "__main__":
    main()

/home/f20210934/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


Using device: cuda
Found existing combined results with completed models: bert, modernbert, deberta, gemini, flair, glove, roberta


===== Skipping flair (already fully processed) =====


===== Skipping glove (already fully processed) =====


===== Skipping roberta (already fully processed) =====


===== Skipping bert (already fully processed) =====


===== Skipping modernbert (already fully processed) =====


===== Skipping deberta (already fully processed) =====
All processing complete. Files saved with suffix: 
Output directories: embeddings_output and variance_plots


<Figure size 864x576 with 0 Axes>

<Figure size 864x576 with 0 Axes>